# 学習の準備
ディープラーニングによりIrisを分類するための準備です。

## ●訓練データとテストデータ
今回はIrisデータセットを訓練データとテストデータに分割します。  
訓練データでニューラルネットワークを訓練し、テストデータでその汎化性能を評価します。  

データセット全体を、どの割合で訓練データとテストデータに分けるのかは、判断に慎重を要する問題です。  
テストデータの割合を増やせば検証の信頼性が高くなりますが、訓練データの割合を小さくしすぎると、学習に必要な訓練データのサンプル数が少なくなってしまいます。  
一般的に、テストデータの割合を20%から30%にするのが妥当な線なのですが、今回は訓練データによる結果とテストデータによる結果をなるべく同等の条件で比較したいので、Irisデータセットのうち半分をテストデータとし、残りを訓練データにします。  

学習中は、訓練データの誤差とテストデータの誤差の変化を記録します。  
また、学習後のニューラルネットワークが正しく品種分類を行えるかどうかの指標として、訓練データとテストデータそれぞれで正解率を測定します。

## ●ネットワーク構成
今回は以下の図で示すニューラルネットワークを構築します。

<img src="images/network.png">

入力層、出力層の間に中間層が2つあり、4層のニューラルネットワークになります。花ごとに4つの測定値があるので、入力層には4つのニューロンを置きます。  
また、今回は入力を3種類の品種に分類するので、出力層には3つのニューロンを置きます。  
これらのニューロンの出力はそれぞれが各品種に対応しており、最も出力の値が大きいニューロンの品種に花が分類されることになります。  
2つの中間層にはとりあえずそれぞれ25個のニューロンを置きます。

## ●学習に関する各種設定
今回は、学習に関する各種設定を以下の通りにします。

|||
|:-:|:-:|
| 中間層の活性化関数 | ReLU |
| 出力層の活性化関数 | ソフトマックス関数 |
| 損失関数 | 交差エントロピー誤差 |
| 最適化アルゴリズム | 確率的勾配降下法 |
| バッチサイズ | 8 |
| 中間層のニューロン数 | 25 |
|||

これまでよりも深い層を持つニューラルネットワークを扱うので、勾配消失に対して頑強なReLUを中間層の活性化関数として使用します。  
また、分類問題を扱うので、出力層の活性化関数にはソフトマックス関数を、損失関数には交差エントロピー誤差を用います。  
そして、最適化アルゴリズムにはとりあえず最もシンプルな確率的勾配降下法を使います。  

今回は、学習を安定させるために、ミニバッチ法を用いてバッチごとに重みとバイアスを更新します。その際のバッチサイズは8に設定します。  
訓練データは1エポックごとにシャッフルされるので、ミニバッチには毎回ランダムなサンプルが含まれることになります。  

上記のケースでは、訓練データのサンプル数は$150 / 2 = 75$なので、1エポックあたりの更新回数は$75 / 8 = 9.375$より端数を切り捨てて9回となります。  
端数の分を学習に用いることも可能ですが、今回はコードをなるべくシンプルに保つため用いません。  

## ●データの入手と前処理
Irisデータセットは、scikit-learnという機械学習用のモジュールを用いて簡単に読み込むことができます。  
scikit-learnはAnacondaに含まれているので、Anacondaを導入済みであれば特に何もしなくても使用することができます。  
以下はscikit-learnを用いてIrisデータセットを読み込み、測定値と正解値を得るためのコードです。  
測定値はニューラルネットワークの入力となります。

In [ ]:
from sklearn import datasets

# -- Irisデータの読み込み --
iris_data = datasets.load_iris()
input_data = iris_data.data
correct = iris_data.target
n_data = len(correct)  # サンプル数

scikit-learnのdatasetsをimportすることにより、`load_iris()`でIrisデータセットを読み込むことができます。  
測定値は`iris_data.data`に格納されているので、これを`input_data`に入れて入力データとします。  
また、品種のインデックスは`iris_data.target`に格納されているので、これを`correct`に入れて正解とします。  
この時点で、`input_data`及び`correct`の中身は以下の通りです。

```
[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 ・・・
 （中略）
 ・・・
 [ 5.9  3.   5.1  1.8]]
```

```
[0 0 ...（中略）... 0 0 1 1 ...（中略）... 1 1 2 2 ...（中略）...2 2]
```

次に、入力データである測定値を標準化します。  
標準化することにより、学習が安定し高速になります。  
標準化されたデータを$z$、元のデータを$X$、元のデータの平均値を$\mu$、元のデータの標準偏差を$\sigma$とすると、標準化は次の式で表されます。

$$ z = \frac{X - \mu}{\sigma} $$

元のデータの各値から平均値を引いて、標準偏差で割ることでデータの平均値は0、標準偏差は1になります。  
この式を元にして、次のコードでIrisデータセットにおける各測定値の標準化を行うことができます。

In [ ]:
# -- 入力データを標準化する --
ave_input = np.average(input_data, axis=0)
std_input = np.std(input_data, axis=0)
input_data = (input_data - ave_input) / std_input

このコードでは、まずNumpyのaverage関数で測定値の平均値を計算します。  
測定値には、Sepal length、Sepal width、Petal length、Petal widthの4種類がありますが、それぞれの種類ごとに平均をとるために、`axis=0`で平均の軸を設定します。  
同様にして標準偏差も計算し、入力データを標準化します。  
`ave_input`と`std_input`はベクトルですが`input_data`は行列なので、ここではブロードキャストを用いた計算が行われます。  

次に、正解値をone-hot表現で表します。出力層のニューロン数が3なので、正解は以下の3通りのone-hot表現で表します。

```
[1 0 0]
[0 1 0]
[0 0 1]
```

Irisデータセットの品種インデックスを、one-hot表現に変換するコードは以下の通りです。

In [ ]:
# -- 正解をone-hot表現にする --
correct_data = np.zeros((n_data, 3))
for i in range(n_data):
    correct_data[i, correct[i]] = 1.0


このコードでは、Numpyのzeros関数で$n \times 3$の行列を作っています。  
そして、各花に対応する行の、品種のインデックスに対応する列の値を1に設定しています。  
これにより、各花の品種がone-hot表現で表され、正解値になります。

`input_data`の中には標準化された値が行列で格納されていますが、各行がそれぞれの花に対応し、各列が各種の測定値に対応します。  `correct_data`の各行も各花に対応し、各列は品種を表すone-hot表現になっています。  
これらのデータ構造を、以下の図に示します。

<img src="images/data_structure.png">

次に、これらのデータを訓練データとテストデータに分割します。  
全てのデータを半分に分けて、それぞれ訓練データとテストデータにします。  
これは、以下のコードで実装できます。

In [ ]:
# -- 訓練データとテストデータ --
index = np.arange(n_data)
index_train = index[index%2 == 0]
index_test = index[index%2 != 0]

input_train = input_data[index_train, :]  # 訓練 入力
correct_train = correct_data[index_train, :]  # 訓練 正解
input_test = input_data[index_test, :]  # テスト 入力
correct_test = correct_data[index_test, :]  # テスト 正解

全部でn個のサンプルがあるのですが、Numpyのarrange関数により0からn-1までが値として格納された配列`index`を作ります。  
そして、このうち2で割った余りが0なもの、すなわち偶数の値を`index_train`に格納して、2で割った余りが0でないのもの、すなわち奇数の値を`index_test`に格納します。  

これらのインデックスとNumpyのスライス機能を用いて、`input_data`と`correct_data`を分割します。  
これらの行列のうち、列のインデックスが`index_train`であるものは`input_train`と`correct_train`に、列のインデックスが`index_test`であるものは`input_test`と`correct_test`に格納します。  
以上により、訓練データとテストデータを用意することができました。